In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e


In [ ]:
!pip install deepchem==2.7.2.dev20231116191348
import deepchem
print(deepchem.__version__)


In [5]:
from rdkit import Chem

In [ ]:
#!git clone https://github.com/NVIDIA/apex
#!cd /content/apex
#!pip install -v --no-cache-dir /content/apex
#ATTEMPT TO INSTALL APEX FEAT LATER
!pip install transformers
!pip install simpletransformers
!pip install wandb
!cd ..

In [7]:
!git clone https://github.com/seyonechithrananda/bert-loves-chemistry.git

Cloning into 'bert-loves-chemistry'...
remote: Enumerating objects: 1566, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1566 (delta 96), reused 92 (delta 91), pack-reused 1364
Receiving objects: 100% (1566/1566), 55.35 MiB | 17.81 MiB/s, done.
Resolving deltas: 100% (1000/1000), done.
Updating files: 100% (122/122), done.


In [8]:
!nvidia-smi #runs on T4 GPU

Mon Jan 22 04:19:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

INSTALLING APEX ON COLAB

In [9]:
import torch
assert torch.cuda.is_available(), "CUDA is not available"


In [ ]:
!pip install -U catalyst
!pip install -U apex


In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./


In [12]:
#Above code block had same error (fix is to restart collab on the same GPU)
from apex import amp

In [13]:
!pip install rdkit

Fine-tuning ChemBERTa on a Small Mollecular Dataset

In [14]:
%cd /content/bert-loves-chemistry

/content/bert-loves-chemistry


In [15]:
!pwd

/content/bert-loves-chemistry


In [16]:
!wget https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/vocab.txt

--2024-01-22 04:20:42--  https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/vocab.txt
Resolving deepchemdata.s3-us-west-1.amazonaws.com (deepchemdata.s3-us-west-1.amazonaws.com)... 52.219.193.130, 52.219.120.241, 52.219.194.34, ...
Connecting to deepchemdata.s3-us-west-1.amazonaws.com (deepchemdata.s3-us-west-1.amazonaws.com)|52.219.193.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3524 (3.4K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>]   3.44K  --.-KB/s    in 0s      

2024-01-22 04:20:42 (177 MB/s) - ‘vocab.txt’ saved [3524/3524]



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read your dataset
df = pd.read_excel('/content/Some_pos_all_neg_DATA.xlsx', sheet_name = 'Multi MODEL DATA')

# Define the proportions for the split
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# Split the dataset
train_df, test_df = train_test_split(df, test_size=(1 - train_ratio), random_state=42)
valid_df, test_df = train_test_split(test_df, test_size=(test_ratio / (test_ratio + validation_ratio)), random_state=42)

# You now have three DataFrames: train_df, validation_df, and test_df
print("train",train_df)
print("valid",valid_df)
print("test",test_df)

In [ ]:
train_df

,CanonicalSMILES,Experimental label,Docking label
254,CC1=C(N=CN=C1OC2CCN(CC2)C(=O)OC(C)C)OC3=C(N=C(...,1,1
1110,C1=C(C=C(C(=C1NC(=O)C2=C(C(=CC(=C2Cl)Cl)Cl)O)O...,0,1
665,CCOC(=O)C(C)(CC1=CSC=N1)C2=CC=NC3=C(C=NN23)C4=...,1,1
806,CCCCCC(CC(=O)CCC1=CC(=C(C=C1)O)OC)O,0,1
715,CC1=C([N+](=O)C2=CC=CC=C2N1[O-])C(=O)C=CC3=CC=...,0,1
...,...,...,...
1130,C(CC(=O)NC(CS)C(=O)NCC(=O)O)C(C(=O)O)N,0,1
1294,CC12CC3(C4CC1(C4(C(O2)O3)COC(=O)C5=CC=CC=C5)OC...,0,0
860,CC(=CCC(C1=CC(=O)C2=C(C=CC(=C2C1=O)O)O)O)C,0,1
1459,CC1CCC2(CCC3(C(=CCC4C3(CCC5C4(CC(C(C5(C)CO)O)O...,0,0


In [ ]:
valid_df

,CanonicalSMILES,Experimental label,Docking label
56,CCC1CC(=O)NN=C1C2=CC3=C(C=C2)N=C(O3)C4=CC(=CC=...,1,1
192,CCC1CC(=O)NN=C1C2=CC3=C(C=C2)N=C(O3)C4=CC=C(C=...,1,1
950,C1=CC(=C(C=C1C=CC(=O)O)O)O,0,1
1196,C1=NC2=C(N1COC(CO)CO)N=C(NC2=O)N,0,0
1455,CC1(C2CCC3(C(C2(CCC1O)C)C(=O)C=C4C3(CCC5(C4CC(...,0,0
...,...,...,...
650,CCOC(=O)C(C)(CC(C)C)C1=CC=NC2=C(C=NN12)C3=CC=C...,1,1
29,C1CCN(CC1)CC2=CC=C(C=C2)C3=NC4=C(O3)C=C(C=C4)C...,1,1
365,CC(CNC(=O)CC1=C(C=CC=C1F)F)C2CCN(CC2)C(=O)OC(C...,1,1
81,CC1=C(C=C(C=C1)C2=NC3=C(O2)C=C(C=C3)C4=NNC(=O)...,1,1


In [ ]:
test_df

,CanonicalSMILES,Experimental label,Docking label
1255,CC1=C(C(=O)C=CO1)O,0,0
405,CS(=O)(=O)C1=CC=C(C=C1)N2CCC(CC2)C3CCN(CC3)C4=...,1,1
792,C1=CC(=CC=C1NC(=O)CCC(=O)O)S(=O)(=O)NC2=NC=CS2,0,1
1067,COC1=C(C=C2C(=C1O)C(=O)C(=CO2)C3=CC=C(C=C3)O)O...,0,1
788,CC(=O)OC1=CC=CC2=CC=CC=C21,0,1
...,...,...,...
705,CC(C)(C)OC(=O)N1CCC(CC1)N(C2CC2)C(=O)C3=CC=C(C...,1,0
566,CC(C)OC(=O)N1CCC(CC1)COC2=CN=C(C=C2)N3CCN(CC3)...,1,1
128,CCC1CC(=O)NN=C1C2=CC3=C(C=C2)N=C(O3)C4=CC=C(C=...,1,1
693,C1CC(C1)C2=NC3=C(CS(=O)(=O)C3)C(=N2)N4CCC(CC4)...,1,0


In [18]:
#LOGGER
from simpletransformers.classification import ClassificationModel
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [19]:
import shutil

# Specify the output directory
output_dir = 'outputs/'

# Clear the contents of the directory
shutil.rmtree(output_dir, ignore_errors=True)


In [20]:
#####OG########
# Import necessary libraries
from simpletransformers.classification import ClassificationModel

# Convert labels to integers
train_df['Experimental label'] = train_df['Experimental label'].astype(int)
train_df['Docking label'] = train_df['Docking label'].astype(int)

valid_df['Experimental label'] = valid_df['Experimental label'].astype(int)
valid_df['Docking label'] = valid_df['Docking label'].astype(int)


# Combining 'Experimental label' and 'Docking label' into a single label
train_df['Combined label'] = train_df['Experimental label'].astype(str) + train_df['Docking label'].astype(str)
valid_df['Combined label'] = valid_df['Experimental label'].astype(str) + valid_df['Docking label'].astype(str)

# Defining labels
labels_list = ['00', '01', '10', '11']

# Initialize the model
model = ClassificationModel(
    'roberta',
    'seyonec/PubChem10M_SMILES_BPE_396_250',
    use_cuda=True,
    num_labels=len(labels_list),
    args={
        'evaluate_each_epoch': True,
        'evaluate_during_training_verbose': True,
        'no_save': True,
        'num_train_epochs': 10,
        'auto_weights': True,
        'early_stopping': True,
        'regression': False,
    }
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE_396_250 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/101k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
#####data aug########

from simpletransformers.classification import ClassificationModel
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

def augment_smiles(smiles, num_augmented_samples):
    mol = Chem.MolFromSmiles(smiles)
    augmented_smiles = [smiles]

    for i in range(num_augmented_samples):
        new_mol = Chem.Mol(mol.ToBinary())
        AllChem.EmbedMolecule(new_mol, AllChem.ETKDG())
        AllChem.UFFOptimizeMolecule(new_mol)
        augmented_smiles.append(Chem.MolToSmiles(new_mol))

    return augmented_smiles

train_df['Augmented_SMILES'] = train_df['CanonicalSMILES'].apply(lambda x: augment_smiles(x, num_augmented_samples=5))
print(train_df)
#flatten
train_df = train_df.explode('Augmented_SMILES')


# labels to integers
train_df['Experimental label'] = train_df['Experimental label'].astype(int)
train_df['Docking label'] = train_df['Docking label'].astype(int)

valid_df['Experimental label'] = valid_df['Experimental label'].astype(int)
valid_df['Docking label'] = valid_df['Docking label'].astype(int)


# Combine 'Experimental label' and 'Docking label' into a single label
train_df['Combined label'] = train_df['Experimental label'].astype(str) + train_df['Docking label'].astype(str)
valid_df['Combined label'] = valid_df['Experimental label'].astype(str) + valid_df['Docking label'].astype(str)

# Defining labels
labels_list = ['00', '01', '10', '11']

# Initialize the model
model = ClassificationModel(
    'roberta',
    'seyonec/PubChem10M_SMILES_BPE_396_250',
    use_cuda=True,
    num_labels=len(labels_list),
    args={
        'evaluate_each_epoch': True,
        'evaluate_during_training_verbose': True,
        'no_save': True,
        'num_train_epochs': 10,
        'auto_weights': True,
        'early_stopping': True,
        'regression': False,  # Ensure you're using classification mode
        'metrics': ["mcc", "tp", "tn", "fp", "fn", "auroc"]
    }
)

In [22]:
print(model.tokenizer)

RobertaTokenizerFast(name_or_path='seyonec/PubChem10M_SMILES_BPE_396_250', vocab_size=7924, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}


In [23]:
#All labels for training aug + og data

import pandas as pd

# Combine 'CanonicalSMILES' and 'Augmented_SMILES' columns
train_df['Combined_SMILES'] = train_df['CanonicalSMILES'] + train_df['Augmented_SMILES']

# Extract unique SMILES and associated labels
unique_data = train_df[['Combined_SMILES', 'Combined label']].drop_duplicates()

# Merge the unique data back with the original DataFrame to get the 'Experimental label' and 'Docking label'
merged_data = pd.merge(unique_data, train_df[['Combined_SMILES', 'Experimental label', 'Docking label']], on='Combined_SMILES', how='left')

# Drop duplicates if any after the merge
merged_data = merged_data.drop_duplicates()

# Reset index for the new DataFrame
merged_data = merged_data.reset_index(drop=True)

# Rename 'Combined_SMILES' to 'CanonicalSMILES'
merged_data = merged_data.rename(columns={'Combined_SMILES': 'CanonicalSMILES'})

# Reorder the columns
train_df = merged_data[['CanonicalSMILES', 'Experimental label', 'Docking label', 'Combined label']]

print(train_df)


                                        CanonicalSMILES  Experimental label  \
0     CC1=C(N=CN=C1OC2CCN(CC2)C(=O)OC(C)C)OC3=C(N=C(...                   1   
1     CC1=C(N=CN=C1OC2CCN(CC2)C(=O)OC(C)C)OC3=C(N=C(...                   1   
2     C1=C(C=C(C(=C1NC(=O)C2=C(C(=CC(=C2Cl)Cl)Cl)O)O...                   0   
3     C1=C(C=C(C(=C1NC(=O)C2=C(C(=CC(=C2Cl)Cl)Cl)O)O...                   0   
4     CCOC(=O)C(C)(CC1=CSC=N1)C2=CC=NC3=C(C=NN23)C4=...                   1   
...                                                 ...                 ...   
2298  CC(=CCC(C1=CC(=O)C2=C(C=CC(=C2C1=O)O)O)O)CCC(C...                   0   
2299  CC1CCC2(CCC3(C(=CCC4C3(CCC5C4(CC(C(C5(C)CO)O)O...                   0   
2300  CC1CCC2(CCC3(C(=CCC4C3(CCC5C4(CC(C(C5(C)CO)O)O...                   0   
2301  C1C(OC(C1O)N2C=NC3=C(N=CN=C32)N)COC1C(OC(C1O)N...                   0   
2302  C1C(OC(C1O)N2C=NC3=C(N=CN=C32)N)CONc1ncnc2c1nc...                   0   

      Docking label Combined label  
0             

In [24]:
# check if our train and evaluation dataframes are setup properly. There should only be two columns for the SMILES string and its corresponding label.
print("Train Dataset: {}".format(train_df.shape))
print("Eval Dataset: {}".format(valid_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

Train Dataset: (2303, 4)
Eval Dataset: (148, 4)
TEST Dataset: (148, 3)


In [25]:
import pandas as pd

with pd.ExcelWriter('combined_label_input_Data.xlsx') as writer:
    train_df.to_excel(writer, sheet_name='Train')
    valid_df.to_excel(writer, sheet_name='Eval')
    test_df.to_excel(writer, sheet_name='Test')

from google.colab import files
files.download('combined_label_input_Data.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("Train Dataset: {}".format(train_df))
print("Eval Dataset: {}".format(valid_df))
print("TEST Dataset: {}".format(test_df))

In [27]:
train_counts = train_df['Combined label'].value_counts()
print("Counts in train_df:\n", train_counts)

valid_counts = valid_df['Combined label'].value_counts()
print("\nCounts in valid_df:\n", valid_counts)


Counts in train_df:
 11    1080
01     706
00     471
10      46
Name: Combined label, dtype: int64

Counts in valid_df:
 11    76
01    45
00    23
10     4
Name: Combined label, dtype: int64


In [28]:
import wandb

wandb.login(key='9049e29a92ea373c0ad75aba6f742e9c1197e110')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [29]:
import shutil
import matplotlib.pyplot as plt

output_dir = '/content/SmilesTokenizer_PubChem_10M_'

shutil.rmtree(output_dir, ignore_errors=True)


In [30]:

# Training and validation metrics
train_loss_list = []
eval_loss_list = []
eval_accuracy_list = []

def track_metrics(labels, preds, **kwargs):
    eval_result = {"train_loss": kwargs.get("train_loss"), "eval_loss": kwargs.get("eval_loss"), "eval_accuracy": kwargs.get("eval_accuracy")}

    # Extract other metrics if needed

    train_loss_list.append(eval_result["train_loss"])
    eval_loss_list.append(eval_result["eval_loss"])
    eval_accuracy_list.append(eval_result["eval_accuracy"])

    print(f"Train Loss: {eval_result['train_loss']}, Eval Loss: {eval_result['eval_loss']}, Eval Accuracy: {eval_result['eval_accuracy']}")




In [ ]:
# Create directory to store model weights (change path accordingly to where you want!)
#!mkdir SmilesTokenizer_PubChem_10M_my_data
import matplotlib as plt
!mkdir SmilesTokenizer_PubChem_10M_a

# Train the model
#model.train_model(train_df, eval_df=valid_df, output_dir='/content/SmilesTokenizer_PubChem_10M_my_data', args={'wandb_project': 'project-name'})
model.train_model(
    train_df,
    eval_df=valid_df,  # Pass the validation set for evaluation during training
    metrics=track_metrics,
    output_dir='/content/SmilesTokenizer_PubChem_10M_a',
    args={
        'wandb_project': 'project-name',
        'evaluate_during_training': True,  # Enable evaluation during training
        'evaluate_during_training_steps': 500,  # Evaluate every 500 steps
    }
)


In [32]:
import sklearn

# accuracy
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

# ROC-PRC
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.average_precision_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/148 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/19 [00:00<?, ?it/s]

Training loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_loss,▂▂▂▇█▇▆▆▆▁▁▁▂▂▂
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,▃▅▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
mcc,▆▇▆▂▁▃▃▃▃███▇▇▇
train_loss,█▅▄▂▁▁▁▁▁▁▁▁▁▁▁
Training loss,4e-05
eval_loss,0.26682
global_step,2880
lr,0.0
mcc,0.89484


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/148 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/19 [00:00<?, ?it/s]

In [33]:
import sklearn

train_result, model_outputs, wrong_predictions = model.eval_model(train_df, acc=sklearn.metrics.accuracy_score, auc=sklearn.metrics.roc_auc_score)

valid_result, model_outputs, wrong_predictions = model.eval_model(valid_df, acc=sklearn.metrics.accuracy_score, auc=sklearn.metrics.roc_auc_score)

print("training results:",train_result)
print("validation results:",valid_result)


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2303 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/288 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/148 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/19 [00:00<?, ?it/s]

training results: {'mcc': 1.0, 'acc': 1.0, 'auc': 1.0, 'eval_loss': 2.3844600137737063e-05}
validation results: {'mcc': 0.8948381385019567, 'acc': 0.9459459459459459, 'auc': 0.9488970588235295, 'eval_loss': 0.2668223686908421}


In [34]:
predictions, raw_outputs = model.predict(['CCCCCCCCC=CCCCCCCCC(=O)NCCC1=CC(=C(C=C1)O)O'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
# Mapping prediction index to corresponding labels
predicted_label = labels_list[predictions[0]]

print("Predicted label:", predicted_label)
print("Raw outputs:", raw_outputs)

predicted_probs = dict(zip(labels_list, raw_outputs[0]))
print("Predicted probabilities for each class:", predicted_probs)

top_class = max(predicted_probs, key=predicted_probs.get)
print("Top predicted class:", top_class)


Predicted label: 00
Raw outputs: [[ 8.8671875  -3.31640625 -2.83398438 -3.00390625]]
Predicted probabilities for each class: {'00': 8.8671875, '01': -3.31640625, '10': -2.833984375, '11': -3.00390625}
Top predicted class: 00


In [39]:
import pandas as pd

# Read the Excel file
df = pd.read_excel('/content/20230404-L2900-Anti-diabetic-Compound-Library-96-well.xlsx', sheet_name='RUN')

smiles = df['SMILES'].tolist()
smiles = [str(s) for s in smiles]
predictions, raw_outputs = model.predict(smiles)
predicted_labels = [labels_list[p] for p in predictions]
df['Predicted label'] = predicted_labels
df.to_excel('/content/Bioactive-Anti-diabetic-Compound-Library_with_predictions_multi.xlsx', index=False)


  0%|          | 0/147 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from IPython.display import display

smiles_list = df['SMILES']
predicted_labels = df['Predicted label']
chemical_names = df['Name']  # Replace 'Name' with the actual column name

filtered_molecules = [(Chem.MolFromSmiles(smiles), name) for smiles, label, name in zip(smiles_list, predicted_labels, chemical_names) if label == '01']

for i, (mol, name) in enumerate(filtered_molecules):
    if mol is not None:
        img = Draw.MolToImage(mol, size=(300, 300))
        display(f"{name}:")
        display(img)


In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from IPython.display import display

smiles_list = df['SMILES']
predicted_labels = df['Predicted label']
chemical_names = df['Name']

filtered_molecules = [(Chem.MolFromSmiles(smiles), name) for smiles, label, name in zip(smiles_list, predicted_labels, chemical_names) if label == '00']

for i, (mol, name) in enumerate(filtered_molecules):
    if mol is not None:
        img = Draw.MolToImage(mol, size=(300, 300))
        display(f"{name}:")
        display(img)
